In [31]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torch.utils.tensorboard as tb
import preprocessing
from torch.utils.data.sampler import SubsetRandomSampler

In [45]:
#get data from preprocessing.py
network_input, network_output = preprocessing.preprocess_notes('C:\\Users\\utkar\\Downloads\\Github Repos\\SonificationML\\Dataset\\Clementi dataset\\Clementi dataset')
network_input, network_output = network_input.cuda(), network_output.cuda()
#print(network_input)
print(network_output)

tensor([ 7, 13, 17,  ...,  1,  7,  1], device='cuda:0')


In [46]:
network_output.max()

tensor(37, device='cuda:0')

In [47]:
# range_value = len(network_input) - len(network_input)%50
# print(range_value)

# network_input = network_input[: range_value]
# network_output = network_output[: range_value]

network_input = network_input[: -3117]
network_output = network_output[: -3117]

print(network_input.shape)
print(network_output.shape)

torch.Size([5500, 50, 38])
torch.Size([5500])


In [35]:
#test_data = np.asarray([[1,2,3,4,5],[2,3,4,5,6],[3,4,5,6,7]])
#test_data = torch.from_numpy(test_data)
#print (test_data)
#print(test_data.shape)

class Stacked_LSTM(nn.Module):
    def __init__(self,input_size, hidden_size, num_layers, output_size):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first = True)
        
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(hidden_size, output_size)
        
        
    def forward(self, x, hidden,batch_size):
        
        output, hidden = self.lstm(x, hidden)        
        #print(output.shape)
        
        # stack up lstm outputs
        output = output.contiguous().view(-1, self.hidden_size)
        #print(output.shape)
        
        output = self.dropout(output)
        output = self.linear(output)
        #print('Linear Output :-',output.shape)
        
        output = F.softmax(output)
        #print('SOFTMAX OUTPUT :--', output)
        
        
        # reshape to be batch_size first
        output = output.view(batch_size, -1)
        #print('Reshape to batch size first :-',output.shape)
        
        output = output[:, -x.shape[2]:] # get last batch of labels
        #print('Final Output :-',output)
        
        #output = output.view(batch_size, -1)
        #print('final output shape',output.shape)
        return output, hidden
    
    def hidden_init(self,batch_size):
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.num_layers, batch_size, self.hidden_size).zero_(),
          weight.new(self.num_layers, batch_size, self.hidden_size).zero_())
        return hidden    

In [36]:
# char2int = {0:0,1:1,2:2,3:3,4:4,5:5,6:6,7:7}
# #print(char2int)

# dict_size = len(char2int)
# seq_len = 5
# batch_size = 3

# def one_hot_encode(sequence, dict_size, seq_len, batch_size):
#     # Creating a multi-dimensional array of zeros with the desired output shape
#     features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
#     # Replacing the 0 at the relevant character index with a 1 to represent that character
#     for i in range(batch_size):
#         for u in range(seq_len):
#             features[i, u, sequence[i][u]] = 1
#     return features

# input_seq = torch.from_numpy(one_hot_encode(test_data, dict_size, seq_len, batch_size))
# #print(input_seq)

In [37]:
#y = torch.from_numpy(np.asarray([[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,1,0],[0,0,0,0,0,0,1,0]]))
#y = torch.tensor([[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,1,0],[0,0,0,0,0,0,1,0]])
#y_new = torch.from_numpy(np.asarray([5,6,6]))

In [38]:
train_size = 0.8
indices = list(range(len(network_input)))
np.random.shuffle(indices)
split = int(np.floor(train_size*len(network_input)))
train_idx, val_idx = indices[:split], indices[split:]

train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)


dataset = TensorDataset(network_input,network_output)
train_loader = DataLoader(dataset, batch_size= 100, sampler=train_sampler)

val_loader = DataLoader(dataset, batch_size= 100,sampler= val_sampler)

In [41]:
import torch.optim as optimizer

train_batch_size = 100
val_batch_size = 100
input_size = 38
hidden_size = 256
num_layer = 2
output_size = 38
clip = 3


model = Stacked_LSTM(input_size,hidden_size,num_layer,output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optimizer.Adam(model.parameters(), lr = 0.1)

#make sure to transfer model to GPU after initializing optimizer
#model.cuda()

In [42]:
len(train_loader)

44

In [44]:
epochs = 5
for e in range(epochs):
    
    train_loss = 0
    val_loss = 0
    val_accuracy = 0
    
    hidden = model.hidden_init(train_batch_size)
    #print(hidden[0].data.shape)
    
    print('hidden[0].shape:- ',hidden[0].shape)
    
    for inputs,labels in train_loader:
        #print(inputs.shape)
        
        #inputs, labels = inputs.cuda(), labels.cuda()
        inputs, labels = inputs, labels

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in hidden])
        

        # zero accumulated gradients
        model.zero_grad()

        # get the output from the model
        output, h = model.forward(inputs, h,train_batch_size)
        #print(output)
        
        #print('Labels Shape :-', (torch.max(labels, 1)[1]).shape)
    
        # calculate the loss and perform backprop
        #loss = criterion(output, torch.max(labels, 1)[1])
        #print('Output :-', output)
        #print('Labels Long :-', labels.long())
        loss = criterion(output,labels.long())

        
        loss.backward()
        
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        train_loss += loss.item()
        #print(train_loss)
              
    model.eval()
    for inputs, labels in val_loader:
        
#       inputs, labels = inputs.cuda(), labels.cuda()
        inputs, labels = inputs, labels

        
        val_h = tuple([each.data for each in hidden])
        
        output, hidden = model.forward(inputs, val_h,val_batch_size)
        loss = criterion(output,labels.long())
        
        val_loss += loss.item()
        
        #calculate accuracy
        top_p, top_class = output.topk(1, dim=1)
        #print('Top Class:- ',top_class)
        equals = top_class == labels.long().view(*top_class.shape)
        #print('Equals:- ', equals)
        val_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
    model.train()
    
    #Averaging losses
    train_loss = train_loss/len(train_loader)
    val_loss = val_loss/len(val_loader)
    val_accuracy = val_accuracy/len(val_loader)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss:{:.6f} \tValidation Accuracy: {:.6f}%'.format(e, train_loss, val_loss, val_accuracy*100))      

hidden[0].shape:-  torch.Size([2, 100, 256])


C:\Users\utkar\Anaconda3\envs\pytorch_env\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


KeyboardInterrupt: 

### Music Genaration
#input_seq

In [ ]:
test_seq = torch.from_numpy(one_hot_encode(test_data, dict_size, seq_len, 1))
test_seq.shape

In [ ]:
#Predicting
test_hidden = model.hidden_init(test_seq.shape[0])
test_output,_ = model.forward(test_seq, test_hidden,test_seq.shape[0])

top_p, top_class = test_output.topk(1,dim =1)
top_class

Using TensorFlow backend.


In [2]:
preprocessing.preprocess_notes()

ValueError: zero-size array to reduction operation maximum which has no identity